#### PIPE - 2:    
This pipe covers the following : 
1. All the features of pipe 1
2. Capturing evidance images
3. Creating CSV file with all detection coordinates
4. Creating CSV file with violation detection coordinates
5. Function to create json based on violation ID for each detected violation done by object

In [5]:
import cv2
import numpy as np
import pandas as pd
import torch
import time
from ultralytics import YOLO
from pathlib import Path
from boxmot import StrongSORT
from utils.utils import *

down = {}
up = {}
text_color = (0, 0, 0)  # Black color for text
yellow_color = (0, 255, 255)  # Yellow color for background
red_color = (0, 0, 255)  # Red color for lines
blue_color = (255, 0, 0)  # Blue color for lines

counter_down = []
counter_up = []

first_boundry_y=170

red_line_y = 198

blue_line_y = 280

offset = 6
# Specify the start and end points of the line
start_point = (300, 198)
end_point = (300, 280)

all_id_tracker=[]
voilation_id_tracker=[]

tracker=tracker_init(cuda_device=torch.cuda.is_available(),cuda_device_number=1)
model=YOLO('yolov8s.pt')
video_path='/media/hlink/hd/test_videos/testx_vid_1.mp4'
vid = cv2.VideoCapture(video_path)
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out_main = cv2.VideoWriter('output_strongs.avi', fourcc, 20.0, (1020, 500))
frame_count=0
while True:    
    ret, im = vid.read()
    
    if not ret:
        break
    
    im=cv2.resize(im,(1020, 500))
    
    
    results=model.predict(im,conf=0.4,verbose=False,device=[1],classes=[2,3])
    im=line_plotter(frame=im,x_start=300,x_end=774,y=198,line_color=red_color,line_name='boundry 1',line_thickness=2,text_color=text_color)
    im=line_plotter(frame=im,x_start=300,x_end=800,y=280,line_color=blue_color,line_name='boundry 2',line_thickness=2,text_color=text_color)
    
    # Specify the color of the line in BGR format (here, it's white)
    color = (255, 255, 255)
    all_frames_record_path=create_directory('all_frames_record')
    violation_frames_record_path=create_directory('all_violation_record')
    
    # Draw the vertical line on the image
    thickness = 2  # You can adjust the thickness as needed
    cv2.line(im, start_point, end_point, color, thickness)
    
    # im=cv2.rectangle(im, (300, 198), (800, 280), (255, 255, 255), 2)
    
    px,conf=prediction_coordinated_hadler(results)
    dets = []
    # Experimenting
    dets=tracker_element_creator(dets,px,conf)        
    
    # print('out_from')
    
    dets = np.array(dets)
    # print(dets)
    if len(dets) > 0:
        tracks = tracker.update(dets, im) # --> M X (x, y, x, y, id, conf, cls, ind)
        # print('tracks',tracks)    
        for track in tracks:
                im,object_id,cx,cy,bbox_list=plot_tracks(track,im)
               
                all_id_tracker.append(object_id)
                # Adding file path for csv
                csv_file_path=detection_coordinate_write(frame_count,object_id,bbox_list,'all_frame_detection_detail.csv')
            
                if red_line_y<(cy+offset) and red_line_y > (cy-offset):
                    # print("entered if 1")
                    down[object_id]=time.time()
            
                if object_id in down:
                    # print("entered if 2")
               
                    if blue_line_y<(cy+offset) and blue_line_y > (cy-offset):
                         # print("entered if 3")
                         elapsed_time=time.time() - down[object_id] 
                         # print("entered if 4")
                         if counter_down.count(object_id)==0:
                            counter_down.append(object_id) 
                            distance = 9
                            est_speed=speed_calculator_kmph(distance,elapsed_time)
                            if est_speed>20:
                                print(f'violation_detected : {object_id}')
                                voilation_id_tracker.append(object_id)
                                violation_frame_writer_op=frame_writer(frame_count,im,violation_frames_record_path)
                                violation_csv_file_path=detection_coordinate_write(frame_count,object_id,bbox_list,'violation_frame_detection_detail.csv')
                                
                            print(f"for {object_id} speed is -> {est_speed}")
    
    all_frame_writer_op=frame_writer(frame_count,im,all_frames_record_path)    
    frame_count+=1
    out_main.write(im)
    
    # # break on pressing q or spaceq
    cv2.imshow('BoxMOT detection', im)     
    # key = cv2.waitKey(25) & 0xFF
    
    if cv2.waitKey(20) & 0xFF == ord('q'):
        break

vid.release()
cv2.destroyAllWindows()
print(all_id_tracker)
print(voilation_id_tracker)

2024-05-06 12:59:04.370 | SUCCESS  | boxmot.appearance.reid_model_factory:load_pretrained_weights:207 - Successfully loaded pretrained weights from "models/osnet_x0_25_msmt17.pt"
/home/hlink/workspace/learning/boxmot/bmt/lib/python3.11/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with an OutOfMemoryError: CUDA out of memory. Tried to allocate 128.00 MiB. GPU  (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:924.)
  return F.conv2d(input, weight, bias, self.stride,


violation_detected : 5
for 5 speed is -> 29
[1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 5, 2, 5, 2, 5, 2, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 9, 5, 9, 5, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 5, 9, 5, 10, 5, 10, 5, 5, 5, 9, 9, 5, 5, 9, 9, 9, 9, 9, 9, 10, 9, 10, 9, 10, 5, 10, 5, 5, 5, 9, 9, 9, 9, 9, 9, 5, 5, 9, 9, 9, 9, 5, 5, 10, 5, 10, 9, 5, 9, 10, 5, 10, 5, 10, 5, 10, 5, 10, 5, 10, 5, 10, 5, 10, 5, 10, 5, 10, 5, 10, 5, 10, 5, 10, 5, 10, 5, 10, 5, 10, 5, 10, 5, 10, 5, 10, 5, 10, 5, 10, 5, 10, 5, 5, 5, 10, 5, 5, 10, 5, 10, 5, 10, 5, 10, 5, 10, 5, 10, 5, 10, 5, 10, 5, 10, 5, 10, 5, 10, 5, 5, 5, 10, 5, 10, 5, 10, 5, 10, 5, 10, 5, 10, 5, 10, 5, 10, 5, 10, 5, 10, 5, 10, 5, 10, 5, 10, 5, 10, 5, 10, 5, 10, 5, 10, 5, 10, 5, 10, 5, 10, 5, 10, 5, 10, 5, 10, 5, 10, 5, 10, 5, 10, 5, 10, 5, 5, 10, 5, 10, 12, 5, 10, 12, 5,

In [5]:
violation_frame_writer_op

'/home/hlink/workspace/learning/boxmot/03_05_2024_infenrence_output/all_violation_record/14755.jpg'

In [2]:
all_id_tracker=set(all_id_tracker)
print(all_id_tracker)

{1, 2, 3, 5, 9, 10, 12, 14, 15, 17, 21, 22, 23, 24, 26, 27, 28, 29, 31, 33, 34, 38, 39, 40, 43, 46, 49, 51, 54, 55, 56, 57, 58, 59, 61, 62, 66, 67, 68, 71, 72, 73, 76, 78, 79, 80, 82, 86, 88, 91, 92, 95, 97, 99, 101, 102, 103, 104, 105, 106, 107, 111, 112, 113, 114, 116, 119, 123, 125, 126, 127, 128, 129, 132, 134, 136, 137, 143, 144, 146, 148, 149, 150, 151, 154, 155, 157, 158, 160, 161, 162, 165, 166, 167, 168, 169, 171, 172, 174, 175, 176, 178, 179, 180, 181, 184, 185, 187, 190, 192, 193, 196, 199, 200, 203, 204, 206, 207, 211, 213, 214, 215, 216, 218, 219, 220, 221, 223, 224, 225, 227, 228, 230, 231, 232, 233, 234, 239, 241, 244, 249, 250, 252, 253, 254, 255, 256, 257, 259, 262, 263, 268, 271, 272, 274, 275, 278, 279, 280, 282, 283, 285, 290, 292, 293, 294, 296, 298, 301, 302, 303, 304, 305, 306, 307, 308, 309, 311, 314, 315, 316, 317, 318, 319, 320, 326, 327, 330, 334, 335, 336, 337, 338, 343, 345, 346, 347, 349, 352, 353, 354, 358, 361, 362, 364, 366, 367, 368, 369, 371, 372, 373

In [2]:
with open(violation_csv_file_path,'r') as file:
    violation_content=file.read()
    print(violation_content)

223,5,484, 237, 581, 319
254,10,463, 223, 570, 329
223,5,484, 237, 581, 319
254,10,463, 223, 570, 329
565,28,558, 250, 598, 305
797,34,476, 256, 500, 301
1573,72,369, 230, 464, 326
1720,88,626, 256, 653, 301
2465,107,356, 222, 452, 329
2869,113,418, 221, 523, 336
3694,174,345, 238, 435, 322
3744,169,465, 239, 558, 319
3820,180,461, 237, 545, 316
4260,213,417, 228, 517, 324
4338,214,437, 236, 535, 322
4357,220,543, 252, 574, 299
4563,223,436, 229, 541, 328
4885,239,332, 239, 420, 323
4930,241,357, 237, 445, 318
5673,274,501, 239, 600, 320
5690,263,393, 230, 494, 324
6562,315,386, 234, 484, 327
6775,320,493, 224, 609, 329
7821,367,490, 238, 593, 321
8777,401,343, 225, 436, 329
9110,422,481, 235, 576, 319
9355,455,566, 254, 596, 301
10309,522,624, 257, 648, 298
10539,530,391, 224, 488, 328
10753,557,473, 232, 580, 325
11594,637,530, 253, 564, 299
12065,650,563, 253, 595, 298
13133,735,533, 255, 562, 301
13201,742,611, 252, 638, 300
14004,794,449, 224, 555, 336
14755,852,467, 224, 578, 328

In [3]:
main_violation_json=voilation_capture_json_creator(voilation_id_tracker,csv_file_path)

In [4]:
%%time
for i in main_violation_json:
    order_maintained=frame_order_checker(main_violation_json[i])
    print(order_maintained)

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
CPU times: user 117 µs, sys: 198 µs, total: 315 µs
Wall time: 316 µs


In [90]:
# %%time
# from concurrent.futures import ThreadPoolExecutor
# with ThreadPoolExecutor(max_workers=1) as executor:
#     for i in main_violation_json:
#         future = executor.submit(frame_order_checker, main_violation_json[i])
#         print(future.result())